# Dimensionality reduction 2

__Week 2 - 3 May 2023__

After we have practised using some dimensionality reduction algorithms in basic mode, we will now turn our attention to arguably the most important part of the process - preprocessing and hyperparameter optimisation. Hopefully this will show how these steps can be crucial in producing the best possible representation of data in lower dimensions. Here, we will make use of a new astronomical dataset with the same algorithms as before (`PCA`, `t-SNE`, `UMAP`). We will end by returning to galaxies in COSMOS2015 and improving on the results from the previous session.

---
### Data

Swift dataset - the dataset is light curves (intensity of light on a timeline) of gamma-ray bursts (GRB) coming from the far reaches of the universe. In addition to the raw data, there are several derived properties:

* `T90_start` & `T90_end` - start and end times of the bursts of light.
* `fluence`
* `hardness`

---
### Exercise

You have now practiced reducing a dataset to a set of basis vectors. The next step is to take an additional step of considering which data it is best to use in the first place and how to prepare it.

* Apply your favourite dimensionality reduction algorithm to: (1) properties, (2) raw observed data, (3) preprocessed observed data in the Swift dataset.

* Tweak hyperparameters of the algorithm - can you find an optimal combination?

* Finally, perform classification with the previous galaxies dataset and plot corresponding ROC curves.

* P.S. Why use only one of the algorithms at a time? Think about using one of these for preprocessing and the other for dimensionality reduction (eg., `PCA` + `t-SNE`).


---
* Authors:  Vadim Rusakov, Charles Steinhardt, Jackson Mann
* Email:  vadim.rusakov@nbi.ku.dk
* Date:   27th of April 2023

In [ ]:
!conda install -c conda-forge umap-learn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import umap
from matplotlib.colors import LogNorm
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA

Load the raw data (light curves), as well as the properties (derived form raw data). 

Make sure to get rid of the `NaN` entries.

In [3]:
# loading data (light curves)
data = pd.read_pickle('datasets/swift_gamma_ray_bursts.zip')

# loading derived properties
props_full = pd.read_csv('datasets/SwiftProperties.csv')

In [17]:
display(data)
display(props_full)
print(np.sum(np.sum(np.isnan(data), axis = 1) == 0))
print(np.sum(np.sum(np.isnan(props_full.values), axis = 1) == 0))

,0,1,2,3,4,5,6,7,8,9,...,51754,51755,51756,51757,51758,51759,51760,51761,51762,51763
GRB160821A,3040.461670,4022.901203,5122.229293,730.993217,1820.487318,6201.249796,2825.393120,-2245.827350,4231.813516,4511.882338,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GRB120728A,13388.589569,6916.159710,26011.608192,15333.879322,1029.069783,25820.420886,25213.990090,-20509.084136,14963.830539,51928.444453,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GRB130529A,146.900355,3855.775433,482.809309,3946.212459,6848.331844,15963.331375,14221.581241,-2863.360659,-6977.527595,5789.405183,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GRB200224A,11782.409301,-80946.096551,25243.726050,36324.085817,69769.124463,71458.766613,-32197.666233,-39477.941367,33453.626408,6539.580672,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GRB070521,1442.752784,4173.589526,-1135.019837,-1629.762578,3105.843840,-923.137123,2412.734935,758.840070,1538.356141,-1031.886713,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GRB170921A,-124.218135,-25821.722524,28321.734750,-1825.126297,23010.675913,29956.132415,-13027.254633,-19645.049125,16164.007062,37024.828956,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GRB200608A,-7310.116100,40779.856477,5897.415396,22256.396092,27992.864330,29697.865841,20076.340872,14487.840183,-2971.811403,4836.721704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GRB070419B,3007.563004,7201.539597,3884.351381,227.146819,2319.663395,3250.195224,3822.244158,1633.088741,1666.129784,7890.267301,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GRB140614B,16172.987845,34668.657976,37024.631188,8796.486937,25617.367415,-42305.084190,44366.560750,29120.597147,8836.179964,3984.667736,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,GRBname,T90_start,T90_end,T100_start,T100_end,fluence,hardness
0,GRB200829A,-74.768,204.076,-62.668,-49.568,1.203408e-04,2.004057
1,GRB200819A,-0.712,30.328,0.312,27.256,4.437842e-07,1.140276
2,GRB200809B,0.000,4.508,0.084,4.284,9.234936e-07,2.254499
3,GRB200806A,-0.132,61.308,3.364,42.052,2.656631e-05,1.855710
4,GRB200729A,0.536,136.536,1.536,123.536,4.187539e-06,2.663482
...,...,...,...,...,...,...,...
1384,GRB041219B,NaN,NaN,14.681,53.785,3.781224e-07,1.159925
1385,GRB041219A,NaN,NaN,NaN,NaN,NaN,NaN
1386,GRB041217,0.824,7.888,1.184,6.852,5.125789e-06,1.522124
1387,GRB200324A,NaN,NaN,NaN,NaN,3.642837e-06,1.359534


1318


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [14]:
df_merged = pd.merge(props_full, data, left_on='GRBname', right_index=True, how='inner')
display(df_merged)

,GRBname,T90_start,T90_end,T100_start,T100_end,fluence,hardness,0,1,2,...,51754,51755,51756,51757,51758,51759,51760,51761,51762,51763
0,GRB200829A,-74.768,204.076,-62.668,-49.568,1.203408e-04,2.004057,2136.449151,4742.107415,7307.372063,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GRB200819A,-0.712,30.328,0.312,27.256,4.437842e-07,1.140276,89786.882904,45610.005944,76758.027888,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,GRB200809B,0.000,4.508,0.084,4.284,9.234936e-07,2.254499,107364.035874,129124.879696,184195.104330,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,GRB200806A,-0.132,61.308,3.364,42.052,2.656631e-05,1.855710,1701.854717,3423.471306,3365.277300,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,GRB200729A,0.536,136.536,1.536,123.536,4.187539e-06,2.663482,11174.821297,-2945.644208,23040.501832,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,GRB041223,-10.536,145.944,1.764,110.824,3.781067e-05,1.840264,1762.756386,2897.753465,3444.927054,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1382,GRB041220,-0.208,6.812,-0.020,5.564,5.876749e-07,1.343941,99854.528414,209384.474307,176684.421948,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1383,GRB041219C,0.000,12.000,0.000,10.000,1.561714e-06,1.044961,44786.689496,29409.994404,23371.116606,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1384,GRB041219B,NaN,NaN,14.681,53.785,3.781224e-07,1.159925,-41589.707460,51938.208369,79090.791765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
# loading data (light curves)
data = pd.read_pickle('datasets/swift_gamma_ray_bursts.zip')

# loading derived properties
props_full = pd.read_csv('datasets/SwiftProperties.csv')

# match & merge properties and raw data tables
df_merged = pd.merge(props_full, data, left_on='GRBname', right_index=True, how='inner')
mask = ~np.isnan(df_merged.iloc[:, 1:]).any(axis=1) # mask of NaNs
df_clean = df_merged.loc[mask] # clean of NaNs
X_data = df_merged.values[mask, 7:] # raw data
X_props = df_merged.values[mask, :7] # derived properties

print(f"Shape of raw data set X_data (nsamples, nfeatures): {X_data.shape}")
print(f"Shape of properties data set X_prop (nsamples, nfeatures): {X_props.shape}")

Shape of raw data set X_data (nsamples, nfeatures): (1313, 51764)
Shape of properties data set X_prop (nsamples, nfeatures): (1313, 7)


## Dataset 1. Gamma-ray bursts

1. Run your favourite algorithm (`t-SNE` or `UMAP`) on the bursts properties (`X_props`). Does it manage to split the samples into two types: eg., short and long bursts? $^1$

2. Can you explain the clumpiness of the reduction? Based on this, do derived properties give a meaningful reduction? In not, can you argue why?

Access the properties for training as: `X_props[:, 1:]`.


$^1$ Hint: short- or long-type of bursts separation is not obvious. Calculate the duration of bursts using the properties table and by subtracting `T90_start` from `T90_end`. Colour your scatter plots in reduced dimensions by duration. Try other properties as well.

In [ ]:
# running t-SNE
tsne = TSNE()
y = tsne.fit_transform()

Below is the code example to make scatter plots, coloured by such properties as `duration`. Try other properties as well.

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')

duration = np.log10(df_clean['T90_end'] - df_clean['T90_start']) # burst duration

sc = ax.scatter(y[:, 0], y[:, 1], s=2.0, norm=LogNorm(), 
                c=duration, cmap='jet')
cbar = plt.colorbar(sc)
cbar.ax.set_ylabel('Duration', rotation=270, labelpad=10)
plt.show()

Reapeat with `UMAP`:

In [ ]:
map = umap.UMAP()
y = map.fit_transform()

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')

sc = ax.scatter(y[:, 0], y[:, 1], s=1.0, norm=LogNorm(), 
                c=duration,  cmap='jet')
cbar = plt.colorbar(sc)
cbar.ax.set_ylabel('Duration', rotation=270, labelpad=10)
plt.show()

3. Now train the algorithm with the raw data `X_data`. Are you able to make cleaner classifications using the projections in the space learnt by the algorithms? If not, do you have some ideas for how to improve this?

To access raw data (light curves) for training, use: `X_data`.

In [ ]:
map = umap.UMAP()
y = map.fit_transform()

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')

duration = np.log10(df_clean['T90_end'] - df_clean['T90_start'])

sc = ax.scatter(y[:, 0], y[:, 1], s=2.0, norm=LogNorm(), 
                c=duration, cmap='jet')
cbar = plt.colorbar(sc)
cbar.ax.set_ylabel('Duration', rotation=270, labelpad=10)
plt.show()

4. Hint: try converting the time-series `X_data` to frequency space by taking a Fourier transform. Repeat the exercise. Does this help to create a cleaner representation in the reduced space? What physical meaning does the separation suggest? (for eg., in terms of the burst duration property).

In [ ]:
# taking Fast Fourier Transform
X_data_fft = abs(np.fft.rfft(X_data)) # take modulus to convert to float from complex data type
mask_fft = ~np.isnan(X_data_fft[:, 1:]).any(axis=1) # NaNs
X_data_mod = X_data_fft[mask_fft] # mask out NaNs

To access the fourier transformed raw data, use: `X_data_mod`:

In [ ]:
# running t-SNE
tsne = TSNE()
y = tsne.fit_transform()

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
duration = np.log10(df_clean['T90_end'] - df_clean['T90_start'])
duration = duration.iloc[mask_fft]

sc = ax.scatter(y[:, 0], y[:, 1], s=2.0, norm=LogNorm(), 
                c=duration, cmap='jet')
cbar = plt.colorbar(sc)
cbar.ax.set_ylabel('Duration', rotation=270, labelpad=10)
plt.show()

5. Try experimenting: the main parameter in `t-SNE` is `perplexity` - try using the values of `5, 10, 15, 20, 50`. The analogous parameter for `UMAP` is `n_neighbours`. Can you find a number that makes an optimal separation of bursts? How would this number have to change depending on the size of the dataset, eg. if you have **10x**, **100x** more samples?

6. Try changing the different options `method='exact'` $\left[ \mathcal{O}(n^2) \right]$ and `method='barnes_hut'` $\left[ \mathcal{O}(n \log{n}) \right]$. The complexity of the operations with the two is different: `barnes_hut` provides an approximate map, but with less time spent.

7. Try different value of the number of iterations (`n_iter`). Can you reach the point where the algorithm converges (be gentle to your computer)? Remember that you need to have at least `n_iter = 250`, as the algorithm requires those for calibration. If it takes too long to converge try taking a smaller sub-sample.

## Dataset 2. COSMOS2015 (galaxies again)

Now, repeat the experiment from the previous session with the galaxies dataset. This time think about preprocessing and hyperparameter optimisation and repeat the exercise. At the end try to classify galaxies as either `alive` or `dead`.

1. Recall that the linear `PCA` before was not capable of linearly separating two types of galaxies at all. In fact, the data you were given were not entirely ready. You need to preprocess them first. For this, take all fluxes and normalize them by one of the other fluxes. $^{1,2}$ Repeat the exercise. Do you get better separation?

$^1$ Hint: it turns out the star forming and dead galaxies are most different in the bluer bands (like `u_flux`).\
$^2$ Hint: plot distributions of features before preprocessing and compare with the same after.

In [ ]:
# turn off warnings, no one needs them...
pd.options.mode.chained_assignment = None  # default='warn'

file = "datasets/cosmos2015.csv"
df = pd.read_csv(file, index_col=False)

# select a random sub-sample of the dataset
n = 10000
idxs = np.arange(df.shape[0])
idxs_rand = np.random.choice(idxs, size=n)
df_cut = df.iloc[idxs_rand] # dataframe
X = df.iloc[idxs_rand].values # array

flux_cols = list(df.columns[4:]) # flux column names
flux_idxs = np.argwhere(np.isin(df.columns, flux_cols)).flatten() # flux column indices

Plot distributions of different fluxes before normalization:

In [ ]:
fig, ax = plt.subplots(1)
for i, col in enumerate(flux_cols):
    vec = X[:, i]
    vec = vec[(vec > (-10)) & (vec < 60)]
    ax.hist(vec, bins=100, label=col, density=True, histtype='step')
ax.set_ylim(0, 0.1)
plt.legend()
plt.show()

Normalize fluxes (as suggested above):

In [ ]:
# normalize fluxes 
norm_flux = df_cut.loc[:, 'u_flux'].values # flux used for normalization
X_trans = np.copy(X)
for col in flux_idxs:
    X_trans[:, col] = # complete the operation: normalize X_trans[:, col]

* Plot distributions of normalized fluxes below. How did the distributions change (in terms of numerical ranges and distribution shapes).

In [ ]:
fig, ax = plt.subplots(1)
for i, col in enumerate(flux_cols):
    vec = X_trans[:, i]
    vec = vec[(vec > (-10)) & (vec < 30)]
    ax.hist(vec, bins=100, label=col, density=True, histtype='step')
ax.set_ylim(0, 0.1)
ax.set_xlim(-10, 60)
plt.legend()
plt.show()

* Access transformed (normalized) data for training, use: `X_trans[:, flux_idxs]`.

In [ ]:
pca = PCA()
y_pcs = pca.fit_transform() # train only on fluxes (raw observed data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlim(np.percentile(y_pcs[:,0], 99), np.percentile(y_pcs[:,0], 1))
ax.set_ylim(np.percentile(y_pcs[:,1], 99), np.percentile(y_pcs[:,1], 1))
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
is_sf = np.isin(df_cut.loc[:, 'is_star_forming'], 1)

ax.scatter(y_pcs[is_sf, 0], y_pcs[is_sf, 1], s=0.02, c='b', norm=LogNorm())
ax.scatter(y_pcs[~is_sf, 0], y_pcs[~is_sf, 1], s=0.02, c='r', norm=LogNorm())
ax.annotate("star forming", xy=(0.05, 0.9), xycoords="axes fraction", 
            color='b', fontsize=12)
ax.annotate("dead", xy=(0.05, 0.86), xycoords="axes fraction", 
            color='r', fontsize=12)
plt.show()

### t-SNE

Repeat with `t-SNE`:

In [ ]:
tsne = TSNE()
y = tsne.fit_transform()

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
is_sf = np.isin(df_cut.loc[:, 'is_star_forming'], 1)

ax.scatter(y[is_sf, 0], y[is_sf, 1], s=0.05, c='b', norm=LogNorm())
ax.scatter(y[~is_sf, 0], y[~is_sf, 1], s=0.05, c='r', norm=LogNorm())
ax.annotate("star forming", xy=(0.05, 0.9), xycoords="axes fraction", 
            color='b', fontsize=12)
ax.annotate("dead", xy=(0.05, 0.86), xycoords="axes fraction", 
            color='r', fontsize=12)
plt.show()

### UMAP

Repeat with `UMAP`:

In [ ]:
map = umap.UMAP()
y = map.fit_transform()

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
is_sf = np.isin(df_cut.loc[:, 'is_star_forming'], 1)

ax.scatter(y[is_sf, 0], y[is_sf, 1], s=0.05, c='b', norm=LogNorm())
ax.scatter(y[~is_sf, 0], y[~is_sf, 1], s=0.05, c='r', norm=LogNorm())
ax.annotate("star forming", xy=(0.05, 0.9), xycoords="axes fraction", 
            color='b', fontsize=12)
ax.annotate("dead", xy=(0.05, 0.86), xycoords="axes fraction", 
            color='r', fontsize=12)
plt.show()

2. Even with a high degree of separation, whether star forming or dead, there is a continuum of galaxies with varying masses and rates of star formation. Therefore, do you ever expect there to be discrete islands of galaxies when you reduce the dimensions? $^1$

$^1$ Hint: produce scatter plots in the reduced space and colour the points by different galaxy properties.

3. Finally, perform classification in maps obtained with different algorithms above. Make decision boundaries with simple `if` statements or even use decision trees or other classifiers.

4. Plot ROC-curves for the above classifications.

In [24]:
# produce ROC curves here

## Bonus: PCA + UMAP

5. Try experimenting: can you combine `PCA` (or `kPCA`) and `t-SNE` as part of the preprocessing and training procedures? Do you get better results?

6. Colouring the points by galaxy properties, what can you learn about active or dead galaxies (eg., which masses or rates of star formation do they tend to have)?

In [ ]:
# preprocessing
### insert your code here ###
# ...
# y_pcs = 

# reducing dimensions to 2 components
### insert your code here ###
# ...
# y = 

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
is_sf = np.isin(df_cut.loc[:, 'is_star_forming'], 1)

ax.scatter(y[is_sf, 0], y[is_sf, 1], s=0.05, c='b', norm=LogNorm())
ax.scatter(y[~is_sf, 0], y[~is_sf, 1], s=0.05, c='r', norm=LogNorm())
ax.annotate("star forming", xy=(0.05, 0.9), xycoords="axes fraction", 
            color='b', fontsize=12)
ax.annotate("dead", xy=(0.05, 0.86), xycoords="axes fraction", 
            color='r', fontsize=12)
plt.show()